# 04. EDS biasing using HTF

Here the collective variable (CV) being biased is the average distance to center of mass.

In [34]:
import hoomd
import hoomd.md
import hoomd.dump
import hoomd.group
import hoomd.htf as htf
import tensorflow as tf
import MDAnalysis as mda
import nglview

In [35]:
#### Build training graph ####

def make_eds_graph(N, NN, set_pt):
    # N= Number of atoms in the system 
    # NN=Number of nearest neighbors 
    # set_pt=set point in EDS method
    graph =htf.graph_builder(NN,output_forces=True)
    #calculate center of mass
    com = tf.reduce_mean(graph.positions[:, :2], 0) 
    #calculate distance of each atom from center of mass
    rs = graph.safe_norm(tf.math.subtract(graph.positions[:, :2], com), axis=1) 
    #calculate the average distance from center of mass. This is the collective variable (CV)
    real_cv = tf.reduce_mean(rs) 
    #calculates the running mean of the CV
    cv_run=graph.running_mean(tensor=real_cv,name='cv_run')
    #calculate the EDS alpha value every 300 steps. 
    eds_alpha = htf.eds_bias(real_cv, set_point=set_pt, period=300,learning_rate=5.0)
    eds_energy = eds_alpha * real_cv #computes EDS energy
    #compute EDS forces
    eds_forces = graph.compute_forces(eds_energy)
    #printer1=tf.Print(cv_run,[cv_run],message='CV')
    #printer2=tf.Print(eds_alpha,[eds_alpha],message='a')    
    graph.save('my_model',force_tensor=eds_forces,virial=None)


In [36]:
#### Hoomd-Sim code ####

make_eds_graph(64, 100, 4.0)

hoomd.context.initialize("--mode=cpu")
with htf.tfcompute('my_model', device='CPU:0') as tfcompute:
    #cut off radius: must be less than the box size
    rcut = 6.0 
    #initialize the lattice
    system = hoomd.init.create_lattice(unitcell=hoomd.lattice.sq(a=2.0),n=[8, 8])
    nlist = hoomd.md.nlist.cell(check_period=1)
    #enable lj pair potential
    lj = hoomd.md.pair.lj(rcut, nlist) 
    #set lj coefficients
    lj.pair_coeff.set('A', 'A', epsilon=1.0, sigma=1.0) 
    hoomd.md.integrate.mode_standard(dt=0.005)
    # set up NVT simulation
    hoomd.md.integrate.nvt(kT=1.0, tau=0.5,group=hoomd.group.all()) 
    #equilibrate
    hoomd.run(3000)
    #simulation
    #nvt_dump = hoomd.dump.gsd(filename='eds_traj.gsd', period=10,group=hoomd.group.all(),  
                              #phase=0, overwrite=True, dynamic=['attribute', 'momentum', 'topology'])
    nvt_dump = hoomd.dump.dcd(filename='eds_traj.dcd', period=10,group=hoomd.group.all(),  
                              phase=0, overwrite=True)
    tfcompute.attach(nlist, r_cut=rcut,save_period=10)
    hoomd.run(15000)

print('Training Done')


Note: Backed-up my_model previous model to my_model/previous_model_16
notice(2): Started TF Session Manager.
notice(2): Group "all" created containing 64 particles
notice(2): -- Neighborlist exclusion statistics -- :
notice(2): Particles with 0 exclusions             : 64
notice(2): Neighbors included by diameter          : no
notice(2): Neighbors excluded when in the same body: no
** starting run **
Time 00:00:00 | Step 3000 / 3000 | TPS 37798.6 | ETA 00:00:00
Average TPS: 37324.6
---------
-- Neighborlist stats:
103 normal updates / 30 forced updates / 0 dangerous updates
n_neigh_min: 0 / n_neigh_max: 30 / n_neigh_avg: 15.3906
shortest rebuild period: 5
-- Cell list stats:
Dimension: 2, 2, 1
n_min    : 10 / n_max: 22 / n_avg: 16
** run complete **
notice(2): Force mode is FORCE_MODE.tf2hoomd 
notice(2): Starting TensorflowCompute 
notice(2): completed reallocate
notice(2): Setting flag indicating virial modification will occur
INFO:tensorflow:The following quantities will computed:
I

## **Now let's visualize**

Required packages in your environment: 
    1. mdanalysis
    2. nglview
    3. moviepy==0.2.2.11
    4. imageio==1.6

**Note**: You can do `pip install nglview` but you might have to do `jupyter-nbextension enable nglview --py --sys-prefix` in your environment

In [55]:
u = mda.Universe('eds_traj.dcd')
print(u)

<Universe with 64 atoms>


In [73]:
view = nglview.show_mdanalysis(u.atoms)
view.background = 'black'

In [77]:
view.add_spacefill()

In [78]:
view.color_by('chainid')

In [ ]:
view